In [1]:
import pandas as pd
import numpy as np
import os
import dotenv
from dotenv import load_dotenv, find_dotenv
import io
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.vectorstores import DocArrayInMemorySearch, FAISS
from langchain_text_splitters import CharacterTextSplitter
from IPython.display import display, Markdown

load_dotenv(find_dotenv())
pd.set_option('display.max_colwidth', None)

In [13]:
def convert_to_float(x):
    try:
        return float(x)
    except (ValueError, TypeError):
        return np.nan

def response_to_df(response):

    response_data = []
    
    for doc in range(len(response)):
        country = response[doc].page_content.split('\n', 8)[0].split(': ', 2)[1]
        title = response[doc].page_content.split('\n', 8)[1].split(': ', 2)[1]
        desc = response[doc].page_content.split('\n', 8)[2].split(': ', 2)[1]
        variety = response[doc].page_content.split('\n', 8)[3].split(': ', 2)[1]
        winery = response[doc].page_content.split('\n', 8)[4].split(': ', 2)[1]
        try:
            points = response[doc].page_content.split('\n', 8)[5].split(': ', 2)[1]
        except:
            points = None
        try:
            price = response[doc].page_content.split('\n', 8)[6].split(': ', 2)[1]
        except:
            price = None
        try:
            taster = response[doc].page_content.split('\n', 8)[7].split(': ', 2)[1]
        except:
            taster = None
    
        response_data.append([
            country,
            title,
            desc,
            variety,
            winery,
            points,
            price,
            taster
        ])
    
        #print(f"response data for {doc}th document", response_data)
    
    df_response = pd.DataFrame(response_data, columns = ['country','title','description','variety','winery','points','price','taster_name'])
    # df_response['points'] = df_response['points'].astype(float)
    # df_response['price'] = df_response['price'].astype(float)
    df_response['points'] = df_response['points'].apply(convert_to_float)
    df_response['price'] = df_response['price'].apply(convert_to_float)
    

    return df_response

In [3]:
key = os.environ.get("OPENAI_API_KEY")

In [4]:
df = pd.read_csv('data/winemag-data-130k-v2.csv')
df.dtypes

Unnamed: 0                 int64
country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

In [5]:
df = df[['country', 'title', 'description', 'variety', 'winery','points','price','taster_name']]

In [6]:
processed_csv = 'data/processed/winemag_130k_slim.csv'
df.to_csv(processed_csv, index=False)

df.head()

,country,title,description,variety,winery,points,price,taster_name
0,Italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",White Blend,Nicosia,87,NaN,Kerin O’Keefe
1,Portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Portuguese Red,Quinta dos Avidagos,87,15.0,Roger Voss
2,US,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",Pinot Gris,Rainstorm,87,14.0,Paul Gregutt
3,US,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Riesling,St. Julian,87,13.0,Alexander Peartree
4,US,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Pinot Noir,Sweet Cheeks,87,65.0,Paul Gregutt


### Leverage FAISS VectorStore with OpenAI Embeddings for Automated Document Retrieval

In [7]:
csv_loader = CSVLoader(file_path=processed_csv)

# create embeddings
embedding = OpenAIEmbeddings(openai_api_key=key)

/var/folders/5_/znw2q2p96zxgvwzl47_6fdsr0000gn/T/ipykernel_20904/3479476841.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=key)


In [8]:
documents = csv_loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# feed full data into CharacterTextSplitter (including Taster, Points, Price, etc)
texts = text_splitter.split_documents(documents)

In [9]:
vectorstore = FAISS.from_documents(texts, embedding)

##### Retrive k results for a question that you want to ask. Essentially, return a large pool of recommendations so we can filter further.

In [10]:
# Let's look for "earthy Italian wines" 
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
response0 = retriever.invoke("Suggest some Italian wines that are earthy, return examples as a table in markdown with columns; country, title, winery, variety, description, points, price, taster.")

In [14]:
df_response = response_to_df(response=response0)

df_response.head(20)

,country,title,description,variety,winery,points,price,taster_name
0,Italy,Vasco Sassetti 2005 Brunello di Montalcino,"Earthy aromas of white mushroom and dried roses that are characteristic of Tuscan Sangiovese are what you notice first here. The wine then segues to fruit and spice aromas, although those berry notes are on the mature and jammy side. Fresh acidity and firm tannins suggest a pairing with red meat.",Sangiovese Grosso,Vasco Sassetti,87.0,NaN,None
1,Italy,SassodiSole 2012 Brunello di Montalcino,"Earthy aromas suggesting grilled porcini, leather, tilled soil and balsam lead the nose. The dense, chewy palate offers mature black cherry, raspberry, licorice and dried sage flavors, set against a tannic backdrop that needs time to soften. A tobacco note signals the close. Drink 2022–2032.",Sangiovese,SassodiSole,90.0,49.0,Kerin O’Keefe
2,Italy,SassodiSole 2012 Brunello di Montalcino,"Earthy aromas suggesting grilled porcini, leather, tilled soil and balsam lead the nose. The dense, chewy palate offers mature black cherry, raspberry, licorice and dried sage flavors, set against a tannic backdrop that needs time to soften. A tobacco note signals the close. Drink 2022–2032.",Sangiovese,SassodiSole,90.0,49.0,Kerin O’Keefe
3,Italy,Bellaria 2011 Assunto (Brunello di Montalcino),"Scorched earth, game, cured meat and leather waft out of the glass. The taught palate offers dried black cherry, pomegranate roasted coffee bean, anise and a vanilla note but not quite enough fruit richness. Tightly wound, grainy tannins leave an astringent finish. Give the tannins a few more years to unwind then drink.",Sangiovese,Bellaria,88.0,NaN,Kerin O’Keefe
4,Italy,Terre da Vino 2008 Essenze (Barolo),"Inky and dark, this has modern touches of toasted oak and exotic spice. Those smoky aromas will decrease with time, giving prominence to the cherry and blackberry flavors already present.",Nebbiolo,Terre da Vino,91.0,60.0,None
5,Italy,Mocavero 2011 Primitivo (Salento),"This earthy wine opens with aromas of ripe plum, tilled earth, game and a whiff of barnyard. The rustic but juicy palate doles out black cherry, spicy blueberry and ground pepper framed by hearty tannins.",Primitivo,Mocavero,86.0,30.0,Kerin O’Keefe
6,Italy,Collosorbo 2010 Riserva (Brunello di Montalcino),"Here's an earthy wine that opens with aromas of scorched earth, leather, truffle, raspberry compote and a note of cured meat. The chewy palate offers mature Morello cherry, grilled herb, coffee, clove and anise while firm, ripe tannins provide the framework. A tobacco note closes the finish. Drink 2018–2028.",Sangiovese,Collosorbo,94.0,75.0,Kerin O’Keefe
7,Italy,Cielo e Terra 2011 Primi Soli Primitivo (Puglia),"A certified organic wine, this offers easy berry aromas of cherry and raspberry that would pair with informal foods like pizza. The mouthfeel is lean and bright.",Primitivo,Cielo e Terra,85.0,NaN,None
8,Italy,Fattoria del Pino 2012 Brunello di Montalcino,"This robust red opens with scorched earth, new leather, underbrush and dark spice aromas. The palate is brawny and monolithic, offering licorice, dried cherry and clove flavors with the warmth of evident alcohol. It's framed by chewy tannins and should be drunk soon to capture the remaining fruit and freshness.",Sangiovese,Fattoria del Pino,88.0,NaN,Kerin O’Keefe
9,Italy,Tabarrini 2009 Colle alle Macchie (Sagrantino di Montefalco),"This brooding wine opens with balsamic aromas of eucalyptus and leather, along with whiffs of freshly turned soil and plum. The palate offers up ripe black cherry, white pepper and the warmth of evident alcohol. It also has massive, astringent tannins. Drink after 2019.",Sagrantino,Tabarrini,89.0,77.0,Kerin O’Keefe


##### Now, let's weight the Recommendations by preferred (theoretical) preference between rating (Points) and price

In [15]:
# Let's say we care more about the rating (75% of our preference)
weight_points = .75
weight_price = .25

df_response['weighted_reccomendation_value'] = (df_response['points'] * weight_points) + -1*(df_response['price'] * weight_price)

df_response.sort_values(by=['weighted_reccomendation_value'], ascending=[False]).head(5)

,country,title,description,variety,winery,points,price,taster_name,weighted_reccomendation_value
17,Italy,Tenuta delle Terre Nere 2008 Etna,"This crisp white wine from Etna offers a bright, luminous hay color and fragrant aromas of green fruit, citrus and white peach. The mouthfeel is creamy but not heavy or thick and those characteristic mineral tones render it unique.",White Blend,Tenuta delle Terre Nere,90.0,18.0,None,63.00
10,Italy,Terra d'Aligi 2009 Pecorino (Terre di Chieti),"Here's a Best Buy white wine that would pair with salads, vegetable dishes and appetizers. It's crisp and luminous with lively aromas of citrus, lemon zest, stone fruit and melon. Pecorino is one of Italy's hottest varieties on the indigenous grape scene.",Pecorino,Terra d'Aligi,86.0,11.0,None,61.75
11,Italy,Terrale 1998 Primitivo (Puglia),"Definitely not Zinfandel-like, despite its Primitivo origins; a rather nondescript wine, as a matter of fact. Light cherry overtones are noted on the nose and palate. The tannins are mouth-drying. This is certainly not a good example of how far Puglia has come with this grape. However, it would work fine with a slice or two of pizza.",Primitivo,Terrale,83.0,5.0,None,61.00
15,Italy,Giuseppe Lonardi 2010 Valpolicella Classico,"There's an earthy, leathery feel to this wine that recalls button mushroom, forest floor and dried rose petal. It shows an informal, compact disposition in the mouth.","Corvina, Rondinella, Molinara",Giuseppe Lonardi,83.0,13.0,None,59.00
5,Italy,Mocavero 2011 Primitivo (Salento),"This earthy wine opens with aromas of ripe plum, tilled earth, game and a whiff of barnyard. The rustic but juicy palate doles out black cherry, spicy blueberry and ground pepper framed by hearty tannins.",Primitivo,Mocavero,86.0,30.0,Kerin O’Keefe,57.00


### More nuanced questions

In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
response2 = retriever.invoke("Suggest the best French wines for desert, return examples as a table in markdown with columns; country, title, winery, variety, description, points, price, taster.")

df_response2 = response_to_df(response=response2)
df_response2

,country,title,description,variety,winery,points,price,taster_name
0,France,Les Vins de Vienne 2010 Les Chirats de Saint-Christophe (Hermitage),"This special wine begins with stunning aromas of cedar, roasted meat and ripe raspberries, then adds savory notes of espresso and black olives to create a harmonious blend of fruity and savory elements. It's full bodied yet supple, with a long, fruit-filled finish. Already enjoyable, it should age well for at least 10 years.",Syrah,Les Vins de Vienne,95.0,120.0,Joe Czerwinski
1,France,Domaine de Pajot 2012 Les Quatre Cépages White (Côtes de Gascogne),"This is a typically light and fruity wine from Gascony. It has light, fresh apple flavors, a green crisp streak and just a touch of spice. Drink now.",White Blend,Domaine de Pajot,84.0,NaN,Roger Voss
2,France,Cave de Tain 2015 Nobles Rives Marsanne (Collines Rhôdaniennes),"Round and pillowy in feel, this is a simple wine, offering modest melon and pineapple fruit. Drink it over the next few months.",Marsanne,Cave de Tain,85.0,13.0,Joe Czerwinski


In [18]:
response3 = retriever.invoke("Suggest the best wines for partying on the boat all day, return examples as a table in markdown with columns; country, title, winery, variety, description, points, price, taster.")

df_response3 = response_to_df(response=response3)
df_response3

,country,title,description,variety,winery,points,price,taster_name
0,France,Grand Bateau 2012 Bordeaux,"Packed with fruit, this is a fine, simple Bordeaux. Red fruits leap out of the glass, allied to light tannins. They make for a wine that's ready to drink young and fresh.",Bordeaux-style Red Blend,Grand Bateau,85.0,15.0,Roger Voss
1,Italy,Tenuta del Buonamico 2011 Vermentino (Montecarlo),"This is the kind of food-friendly wine to pair with grilled shrimp kebabs or fish tacos. It presents clean aromas of citrus, pear and stone fruit, backed by zesty acids on the finish.",Vermentino,Tenuta del Buonamico,87.0,16.0,None
2,Italy,Cantina di Soave 2011 Volére Rosato (Delle Venezie),"This bright pink rosato opens with touches of apple skin and citrus, especially pink grapefruit. This exceedingly informal box wine could work at picnics or pool parties.",Rosato,Cantina di Soave,82.0,15.0,None


In [20]:
response4 = retriever.invoke("Suggest the best Australian wines for pairing with cheese, return examples as a table in markdown with columns; country, title, winery, variety, description, points, price, taster.")

df_response4 = response_to_df(response=response4)
df_response4

,country,title,description,variety,winery,points,price,taster_name
0,Australia,Wolf Blass 2001 Chardonnay (South Australia),"A simple but good wine, with toast and golden apple flavors. Has nice apple and olive oil aromas, and a mouth-puckering herb and white pepper finish.",Chardonnay,Wolf Blass,84.0,12.0,None
1,Australia,Vasse Felix 2012 Cabernet Sauvignon (Margaret River),"This crisp, medium-bodied wine showcases Cabernet's hallmark cassis fruit. Sure, there are hints of cranberry, raspberry and cherry, but black currant defines this wine, running from start to finish. Dusty tannins swirl gently on the palate, combining with tart acidity to offer just the right amount of structure. Drink now–2025.",Cabernet Sauvignon,Vasse Felix,90.0,40.0,Joe Czerwinski
2,Australia,Wolf Blass 2010 White Label Chardonnay (Adelaide Hills),"When a producer renowned for its red wines can turn out an elegant, understated white of this quality, you know Australian Chardonnay has firmly arrived. Subtle accents of toasted nuts frame citrusy fruit, and while the wine is full-bodied and round in the mouth, it remains focused and long and on the finish. If you're fortunate enough to get some of the 600 bottles imported, drink the wine over the next couple of years.",Chardonnay,Wolf Blass,91.0,32.0,Joe Czerwinski
